# Analysis of the correlation between Housing Sales Prices and Venues in the city of Toronto

Download and import the House Sales in Ontario dataset from https://www.kaggle.com/mnabaee/ontarioproperties/downloads/ontarioproperties.zip/1

In [1]:
import pandas as pd

df = pd.read_csv("properties.csv")
df=df.drop(df.columns[0], axis=1)
df.columns=['Address','Area','Price','Latitude','Longitude']
df.head()

,Address,Area,Price,Latitude,Longitude
0,"86 Waterford Dr Toronto, ON",Richview,999888.0,43.679882,-79.544266
1,"#80 - 100 BEDDOE DR Hamilton, ON",Chedoke Park B,399900.0,43.250000,-79.904396
2,"213 Bowman Street Hamilton, ON",Ainslie Wood East,479000.0,43.251690,-79.919357
3,"102 NEIL Avenue Hamilton, ON",Greenford,285900.0,43.227161,-79.767403
4,"#1409 - 230 King St Toronto, ON",Downtown,362000.0,43.651478,-79.368118


Filter out entries in the city of Toronto

In [2]:
df_toronto = df[df['Address'].str.contains("Toronto")].reset_index(drop=True)
df_toronto.head()

,Address,Area,Price,Latitude,Longitude
0,"86 Waterford Dr Toronto, ON",Richview,999888.0,43.679882,-79.544266
1,"#1409 - 230 King St Toronto, ON",Downtown,362000.0,43.651478,-79.368118
2,"254A Monarch Park Ave Toronto, ON",Old East York,1488000.0,43.686375,-79.328918
3,"532 Caledonia Rd Toronto, ON",Fairbank,25.0,43.691193,-79.461662
4,"47 Armstrong Ave Toronto, ON",Wallace Emerson,113.0,43.664101,-79.439751


Get the mean price for the various areas across Toronto

In [3]:
df_toronto = df_toronto.groupby('Area').mean().reset_index()
df_toronto.head()

,Area,Price,Latitude,Longitude
0,Agincourt,4.290126e+05,43.788408,-79.278037
1,Agincourt North,2.200000e+06,43.803215,-79.242554
2,Alderwood,9.739375e+05,43.603299,-79.545057
3,Amesbury,7.945000e+04,43.704548,-79.482700
4,Armdale,5.913167e+04,43.828528,-79.251296


Get the geographical coordinates of Toronto

In [4]:
from geopy.geocoders import Nominatim 

address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Ontario are 43.653963, -79.387207.


Visualize the areas in Toronto 

In [5]:
import folium

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Area']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Remove the entry of Sunnyside as it seems to be incorrect

In [6]:
df_toronto=df_toronto[df_toronto.Area != 'Sunnyside']

Define Foursquare Credentials and Version

In [7]:
CLIENT_ID = 'TXYVVJKZPS5W3PBAZHSGR0D1SKZDSH453MMPQJV3PZ2RWFW2' 
CLIENT_SECRET = '4PMZIIPMVEQYJCZZ2PEDNSHET53CWCSLAPEXRNLYBHPOSHFD' 
VERSION = '20180605' 

Create a function to get all nearby venues for all the neighborhoods in Toronto

In [8]:
import requests

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
    
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [9]:
LIMIT = 100
RADIUS = 500
toronto_venues = getNearbyVenues(names=df_toronto['Area'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude']
                                  )


In [10]:
print(toronto_venues.shape)
toronto_venues.head()

(4038, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agincourt,43.788408,-79.278037,One2 Snacks,43.787048,-79.276658,Asian Restaurant
1,Agincourt,43.788408,-79.278037,In Cheon House Korean & Japanese Restaurant 인천관,43.786468,-79.275693,Korean Restaurant
2,Agincourt,43.788408,-79.278037,Tim Hortons,43.785637,-79.279215,Coffee Shop
3,Agincourt,43.788408,-79.278037,Maple Yip Seafood 陸羽海鮮酒家,43.784752,-79.277787,Chinese Restaurant
4,Agincourt,43.788408,-79.278037,Beef Noodle Restaurant 老李牛肉麵,43.785937,-79.276031,Chinese Restaurant


Encode all the venue categories for each neighborhood

In [11]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

cols = list(toronto_onehot)
cols.insert(0, cols.pop(cols.index('Neighborhood')))
toronto_onehot = toronto_onehot.ix[:, cols]


toronto_onehot.head()

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys


,Neighborhood,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Aquarium,Arcade,...,Video Store,Vietnamese Restaurant,Vineyard,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Agincourt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Agincourt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Agincourt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Agincourt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [12]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Aquarium,Arcade,...,Video Store,Vietnamese Restaurant,Vineyard,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Agincourt North,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Alderwood,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Amesbury,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Armdale,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Create a new dataframe to display the top 10 venues for each neighborhood

In [13]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [14]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Restaurant,Shopping Mall,Coffee Shop,Vietnamese Restaurant,Hong Kong Restaurant,Korean Restaurant,Rental Car Location,Cantonese Restaurant,Asian Restaurant
1,Agincourt North,Furniture / Home Store,American Restaurant,Coffee Shop,Yoga Studio,Flea Market,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop
2,Alderwood,Pizza Place,Pub,Gym,Sandwich Place,Coffee Shop,Dance Studio,Pharmacy,Skating Rink,Pool,Fast Food Restaurant
3,Amesbury,Gym / Fitness Center,Park,Coffee Shop,Athletics & Sports,Restaurant,Hobby Shop,Filipino Restaurant,Event Space,Falafel Restaurant,Farmers Market
4,Armdale,Indian Restaurant,Sports Bar,Badminton Court,Skating Rink,Grocery Store,Flea Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop


#### Cluster Neighborhoods

Normalize the house prices

In [15]:
df_normal_price =df_toronto[['Area','Price']].copy()
df_normal_price['Price']=df_normal_price['Price']/df_normal_price['Price'].max()
df_normal_price.columns=['Neighborhood','Price']
df_normal_price.head()

,Neighborhood,Price
0,Agincourt,0.030023
1,Agincourt North,0.153957
2,Alderwood,0.068157
3,Amesbury,0.005560
4,Armdale,0.004138


Create with new dataframe with venue and price data

In [16]:
toronto_grouped_clustering = toronto_grouped.join(df_normal_price.set_index('Neighborhood'), on='Neighborhood')
del toronto_grouped_clustering['Neighborhood']

toronto_grouped_clustering.head()

,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,...,Vietnamese Restaurant,Vineyard,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Price
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.030023
1,0.0,0.0,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.153957
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.068157
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005560
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.004138


#### Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters

In [23]:
from sklearn.cluster import KMeans

kclusters = 4
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

Create a new dataframe that adds the cluster, geographical data and mean price to the top 10 venues for each neighborhood

In [25]:
del neighborhoods_venues_sorted['Cluster Labels']
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto
toronto_merged.columns=['Neighborhood','Price','Latitude','Longitude']
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() 

,Neighborhood,Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,4.290126e+05,43.788408,-79.278037,3.0,Chinese Restaurant,Restaurant,Shopping Mall,Coffee Shop,Vietnamese Restaurant,Hong Kong Restaurant,Korean Restaurant,Rental Car Location,Cantonese Restaurant,Asian Restaurant
1,Agincourt North,2.200000e+06,43.803215,-79.242554,3.0,Furniture / Home Store,American Restaurant,Coffee Shop,Yoga Studio,Flea Market,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop
2,Alderwood,9.739375e+05,43.603299,-79.545057,2.0,Pizza Place,Pub,Gym,Sandwich Place,Coffee Shop,Dance Studio,Pharmacy,Skating Rink,Pool,Fast Food Restaurant
3,Amesbury,7.945000e+04,43.704548,-79.482700,3.0,Gym / Fitness Center,Park,Coffee Shop,Athletics & Sports,Restaurant,Hobby Shop,Filipino Restaurant,Event Space,Falafel Restaurant,Farmers Market
4,Armdale,5.913167e+04,43.828528,-79.251296,3.0,Indian Restaurant,Sports Bar,Badminton Court,Skating Rink,Grocery Store,Flea Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop


Check to see if any rows are null and delete those rows

In [26]:
toronto_merged[toronto_merged.isnull().any(axis=1)]

,Neighborhood,Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
136,Palmerston,668500.0,43.743946,-80.12939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
toronto_merged=toronto_merged[toronto_merged.Neighborhood != 'Palmerston']

In [28]:
toronto_merged['Cluster Labels']=toronto_merged['Cluster Labels'].astype(int)

Visulize the clusters

In [38]:
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
           
map_clusters

### Examine the clusters

In [30]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

#### Cluster 1

In [39]:
cluster1 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
print('1st Most Common Venue(s)',cluster1['1st Most Common Venue'].mode())
print('2nd Most Common Venue(s)',cluster1['2nd Most Common Venue'].mode())
print('Average Price',cluster1.Price.mean())
print('The number of rows are',cluster1.shape[0])

1st Most Common Venue(s) 0    Park
dtype: object
2nd Most Common Venue(s) 0    Park
dtype: object
Average Price 1202752.597849496
The number of rows are 22


#### Cluster 2

In [40]:
cluster2 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
print('1st Most Common Venue(s)',cluster2['1st Most Common Venue'].mode())
print('2nd Most Common Venue(s)',cluster2['2nd Most Common Venue'].mode())
print('Average Price',cluster2.Price.mean())
print('The number of rows are',cluster2.shape[0])

1st Most Common Venue(s) 0    Tennis Court
dtype: object
2nd Most Common Venue(s) 0    Yoga Studio
dtype: object
Average Price 2570911.9404761903
The number of rows are 4


#### Cluster 3

In [41]:
cluster3 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
print('1st Most Common Venue(s)',cluster3['1st Most Common Venue'].mode())
print('2nd Most Common Venue(s)',cluster3['2nd Most Common Venue'].mode())
print('Average Price',cluster3.Price.mean())
print('The number of rows are',cluster3.shape[0])

1st Most Common Venue(s) 0    Pizza Place
dtype: object
2nd Most Common Venue(s) 0    Pizza Place
1    Wings Joint
dtype: object
Average Price 679061.2439325323
The number of rows are 15


#### Cluster 4

In [42]:
cluster4 = toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
print('1st Most Common Venue(s)',cluster4['1st Most Common Venue'].mode())
print('2nd Most Common Venue(s)',cluster4['2nd Most Common Venue'].mode())
print('Average Price',cluster4.Price.mean())
print('The number of rows are',cluster4.shape[0])

1st Most Common Venue(s) 0    Coffee Shop
dtype: object
2nd Most Common Venue(s) 0    Coffee Shop
dtype: object
Average Price 876866.1095713675
The number of rows are 173
